In [ ]:
# Get configuration path
from os.path import join
examples_folder = '../../crosscompute-examples/reports'
configuration_folder = examples_folder + '/randomize-histograms'
configuration_path = join(configuration_folder, 'serve.yml')
configuration_path

In [ ]:
# Load configuration
import yaml
d = automation_configuration = yaml.safe_load(open(configuration_path, 'rt'))
d

In [ ]:
# Get input variable definitions
input_variable_definitions = d['input']['variables']
input_variable_definitions

In [ ]:
batch_definition = d['batches'][0]
batch_definition

In [ ]:
relative_batch_folder = batch_definition['folder']
relative_batch_folder

In [ ]:
from os.path import join
batch_folder = join(configuration_folder, relative_batch_folder)
batch_folder

In [ ]:
ls $batch_folder

In [ ]:
# _ input_batch_folder
# batch_input_folder

input_folder = join(batch_folder, 'input')
input_folder

In [ ]:
ls $input_folder

In [ ]:
cat $input_folder/variables.json

In [ ]:
relative_variable_path = 'variables.json'

In [ ]:
variable_path = join(input_folder, relative_variable_path)
variable_path

In [ ]:
import json
data_dictionary = json.load(open(variable_path, 'rt'))
data_dictionary

In [ ]:
from collections import defaultdict
variable_definitions_by_path = defaultdict(list)
for d in input_variable_definitions:
    path = d['path']
    variable_definitions_by_path[path].append(d)
variable_definitions_by_path = dict(variable_definitions_by_path)
variable_definitions_by_path

In [ ]:
batch_configuration = {'input': {'variables': []}, 'output': {'variables': []}}
batch_configuration

In [ ]:
# Set input variable data
for relative_path, variable_definitions in variable_definitions_by_path.items():
    # TODO: Check path extension
    path = join(input_folder, relative_path)
    data = json.load(open(path, 'rt'))
    for variable_definition in variable_definitions:
        variable_id = variable_definition['id']
        variable_data = data[variable_id]
        
        batch_configuration['input']['variables'].append({
            'id': variable_id,
            'data': variable_data,
        })
batch_configuration

In [ ]:
# Load input template -- we have no template! so we render the default form
automation_configuration['input']

In [ ]:
automation_configuration['input']['variables']

In [ ]:
input_variable_ids = [_['id'] for _ in automation_configuration['input']['variables']]
input_variable_ids

In [ ]:
timeit '{}'.format(1)

In [ ]:
timeit '%s' % 1

In [ ]:
x = 1

In [ ]:
timeit f'{x}' 

In [ ]:
x = 'a'

In [ ]:
timeit '{' + x + '}'

In [ ]:
['{' + _ + '}' for _ in input_variable_ids]

In [ ]:
template_markdown = ' '.join('{' + _ + '}' for _ in input_variable_ids)
template_markdown

In [ ]:
import re
re.sub(r'a', 'b', 'apple apple')

In [ ]:
re.search(r'{\s*mean\s*}', '{ mean }')

In [ ]:
variable_definitions = automation_configuration['input']['variables']
variable_definitions

In [ ]:
variable_id = variable_definitions[0]['id']
variable_id

In [ ]:
variable_id_pattern = re.compile(r'{\s*' + variable_id + r'\s*}')
variable_id_pattern

In [ ]:
# if variable_id_pattern.search(template_markdown):
random_placeholder = 'HTML 123123'
variable_id_pattern.sub(random_placeholder, template_markdown)

In [ ]:
batch_configuration['input']['variables']

In [ ]:
batch_variable_definitions = batch_configuration['input']['variables']
variable_data_by_id = {_['id']: _['data'] for _ in batch_variable_definitions}
variable_data_by_id

In [ ]:
# maybe we do not need to merge dictionaries

In [ ]:
assert variable_definitions[0]['view'] == 'number'
variable_id = 'mean'
variable_data = variable_data_by_id[variable_id]
variable_html = "<input class='input {variable_id}' type='number' value='{variable_data}'>".format(
    variable_id=variable_id,
    variable_data=variable_data)
variable_html

In [ ]:
import re

number_mapping = {'1': 'one321',
                  '2': 'two',
                  '3': 'three'}
s = "1 testing 2 3"

re.sub(r'\d', lambda x: number_mapping[x.group()], s)

In [ ]:
def f(match):
    t = match.group()
    print(t)
    return t

re.sub(r'\d', f, '1 2 3')

In [ ]:
# Find matching input variable ids in input template

In [ ]:
# Render html from variable definition

In [ ]:
VARIABLE_ID_PATTERN = re.compile(r'{\s*([^}]+?)\s*}')
match = VARIABLE_ID_PATTERN.match('{ }}')
match

In [ ]:
match.group(0)

In [ ]:
# Substitute html into template
def f(match):
    matching_text = match.group(0)
    variable_id = match.group(1)
    
    try:
        variable_data = variable_data_by_id[variable_id]
    except KeyError:
        return matching_text
    
    replacement_text = "<input class='input {variable_id}' type='number' value='{variable_data}'>".format(
        variable_id=variable_id,
        variable_data=variable_data)
    return replacement_text

rendered_markdown = VARIABLE_ID_PATTERN.sub(f, template_markdown)
rendered_markdown

In [ ]:
# Render template into html

In [ ]:
from markdown import markdown
html = markdown(rendered_markdown)
html

In [ ]:
open('x.html', 'wt').write(html)

In [ ]:
# Turn template from markdown to html